# 600 Times 을 분할하여 파생변수 생성 
600 Times을 Aggregation 하여 학습한 결과, 각 id별 한 개의 데이터만 존재.    
중복되는 label이 있지만, 데이터셋이 부족하여 과적합의 가능성이 높음.   
마지막 submission 을 위해 예측할 때 사용할 test 데이터셋의 shape을 고려해야함.  

1. 3125 * 600 = 1,875,000 개 
    - 모든 데이터셋 전체 학습 
2. 3125 * ( 600 / 100 ) = 18,750 개 
    - 100 타임별로 분할하여 max, min, mean 계산하여 데이터셋 구성
3. 3125 * ( 600 / 50 ) = 37,500 개 
    - 50 타임별 분할하여 max, min, mean 계산하여 데이터셋 구성
4. 3125 * ( 600 / 10 ) = 187,500 개
    - 10 타임별 분할하여 max, min, mean 계산하여 데이터셋 구성


In [ ]:
start = time.time()
sec = time.time()-start
times = str(datetime.timedelta(seconds = sec)).split('.')

In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import time
import datetime
pd.set_option('display.max_columns', None)

In [2]:
train = pd.read_csv('train_features.csv')
test = pd.read_csv('test_features.csv')
train_label = pd.read_csv('train_labels.csv')
submission = pd.read_csv('sample_submission.csv')

In [29]:
ids = train.id.nunique()

In [30]:
df = []

In [34]:
for i in range(ids):
    df.append(train.iloc[600*i : 600*i + 600])

In [39]:
n = 1 
df[10].iloc[n*100:n*100+100].drop('time',axis = 1, inplace = False).groupby('id').agg(['max','min','mean'])

acc_x                         acc_y                        acc_z  \
         max       min      mean       max       min      mean      max   
id                                                                        
10  0.013193 -0.881252 -0.463381 -0.468754 -0.948012 -0.750204  0.57077   

                             gy_x                             gy_y             \
         min      mean        max        min       mean        max        min   
id                                                                              
10 -0.286108  0.285101  84.239602 -85.502858  13.611083  69.527339 -140.60047   

                   gy_z                         
        mean        max         min       mean  
id                                              
10 -8.824838  57.267105 -128.040279 -16.729449

In [85]:
df_id = pd.DataFrame()

for i in range(3):
    tmp = df[10].iloc[i*200:i*200+200].drop('time',axis = 1, inplace = False).groupby('id').agg(['max','min','mean'])
    # tmp.reset_index(inplace = True)
    df_id = pd.concat([df_id,tmp], axis = 1 )

In [86]:
df_id

acc_x                         acc_y                        acc_z  \
         max       min      mean       max       min      mean      max   
id                                                                        
10  0.013193 -1.366979 -0.694201 -0.036306 -1.004319 -0.536356  0.57077   

                              gy_x                               gy_y  \
         min      mean         max         min       mean         max   
id                                                                      
10 -0.304365  0.179037  110.085425 -128.561551 -16.647662  112.403036   

                              gy_z                           acc_x            \
          min      mean        max         min      mean       max       min   
id                                                                             
10 -140.60047 -2.487432  63.325123 -128.040279 -9.301269  0.800592 -0.280801   

                 acc_y                         acc_z                      \
        mean       max       min      mean       max       min      mean   
id                                                                         
10  0.358864 -0.674516 -1.146763 -0.864957  0.534329 -0.378424  0.043128   

         gy_x                            gy_y                       \
          max         min     mean        max        min      mean   
id                                                                   
10  88.849273 -105.346273 -2.24919  47.366107 -64.348864 -3.126423   

          gy_z                         acc_x                         acc_y  \
           max       min      mean       max       min      mean       max   
id                                                                           
10  115.249587 -94.87906 -3.946249  0.010429 -1.192412 -0.594965  0.010201   

                           acc_z                           gy_x             \
         min      mean       max       min      mean        max        min   
id                                                                           
10 -0.970939 -0.620634  0.577577 -0.244272  0.229488  104.78766 -64.642513   

                     gy_y                             gy_z                      
         mean         max        min       mean        max        min     mean  
id                                                                              
10  24.109274  114.751808 -50.167472  12.156522  85.943547 -77.133264  5.56261

In [88]:
df_id.shape

(1, 54)

In [90]:
df_agg2 = pd.DataFrame()

for j in range(ids):
    df_agg = pd.DataFrame()
    for i in range(3):
        tmp = df[j].iloc[i*200 : i*200 + 200].drop('time', axis = 1, inplace = False).groupby('id').agg(['max','min','mean'])
        df_agg = pd.concat([df_agg, tmp], axis = 1)
        
    df_agg2 = df_agg2.append(df_agg)

In [91]:
df_agg2

acc_x                         acc_y                         acc_z  \
           max       min      mean       max       min      mean       max   
id                                                                           
0     1.343113  0.613582  0.939629  0.133165 -0.624113 -0.240798 -0.054876   
1    -0.059166 -2.156208 -0.541579  0.495337 -0.997057 -0.413170  0.861091   
2     1.219836 -1.142847  0.206800  0.469709 -0.690528 -0.102568  1.088743   
3    -0.679208 -1.339996 -0.918718  0.210985 -0.540827 -0.173210  0.410242   
4     0.599720 -0.831264 -0.079400  0.220760 -2.055076 -0.905563  1.057852   
...        ...       ...       ...       ...       ...       ...       ...   
3120  0.390798 -1.624711 -0.288550 -0.027536 -1.289257 -0.681382  0.849966   
3121 -0.446650 -1.575455 -0.981668  0.117965 -0.600138 -0.192524  0.176047   
3122  0.744666 -2.141936 -1.095197  1.268138 -2.036646 -0.414260  0.803950   
3123  0.164426 -0.929133 -0.316117  1.471337  0.397902  0.885078  0.382916   
3124  0.538809 -0.980186 -0.275555 -0.305105 -0.891517 -0.667109  0.543977   

                                gy_x                               gy_y  \
           min      mean         max         min       mean         max   
id                                                                        
0    -0.672738 -0.344088   30.962266  -46.254836  -2.085009   69.847244   
1    -0.522313  0.313459  176.742984 -322.482025  -3.292411  221.240277   
2     0.073846  0.558121   73.525082 -127.845394  -5.228462  202.876776   
3    -0.181537  0.148455  133.050586  -69.419166  14.740780  141.001242   
4    -1.231836  0.186689  140.227116 -356.302649 -16.625429  284.952954   
...        ...       ...         ...         ...        ...         ...   
3120 -0.551185  0.194684  144.269756 -194.369980 -18.895973  134.585268   
3121 -0.403201 -0.107707  155.958950  -28.736317  35.088227   48.465535   
3122 -0.459267  0.278428  400.662632 -731.029790 -10.348042  342.797555   
3123 -0.646863 -0.132612  168.716922 -121.756774 -12.642361  198.033474   
3124 -0.163281  0.312278   31.017538  -33.240136  -5.343389   88.243786   

                                   gy_z                            acc_x  \
             min       mean         max         min       mean       max   
id                                                                         
0     -85.887677  -2.103937   55.953827  -65.593383   5.986721  1.284556   
1    -315.096003  10.504189   93.343216 -179.514208  19.414557  0.316553   
2    -228.305082 -10.437496   55.642836  -41.230252   1.949360  1.003180   
3     -82.537304   0.312905   56.456908  -85.600536  -2.003227 -0.622250   
4     -42.284998  24.280372  162.734781 -171.614590  10.553543 -0.335596   
...          ...        ...         ...         ...        ...       ...   
3120 -190.818371  -6.619741  121.958427  -70.429209  16.932698 -0.068120   
3121  -33.757220   6.868583   18.956690  -39.777626  -4.569780 -0.608541   
3122 -350.803519  -9.480516  453.943910 -167.145052  21.130014  0.023353   
3123 -148.431769  10.955653  120.963282 -206.580638   1.715393  0.482880   
3124 -134.268304 -15.040502   66.437469  -83.882730 -12.614084  0.534176   

                             acc_y                         acc_z            \
           min      mean       max       min      mean       max       min   
id                                                                           
0     0.600915  0.926688  0.176871 -0.618396 -0.223190 -0.124806 -0.754574   
1    -2.090578 -0.924799  0.228237 -1.295598 -0.424058  0.682967 -0.923339   
2    -1.058880 -0.223230  0.650645 -0.690990 -0.058375  1.332992  0.182810   
3    -1.097469 -0.795698  0.218557 -0.175634  0.010345  0.598814  0.037733   
4    -2.429109 -0.927882  1.724782 -1.176536 -0.166780  2.130765 -1.250483   
...        ...       ...       ...       ...       ...       ...       ...   
3120 -1.256543 -0.519626  0.168070 -0.920347 -0.580878  0.618591 -0.399849   
3121 -1.316488 -0.96

In [94]:
test_ids = test.id.nunique()
test_ids

782

In [98]:
df_test = []
for i in range(test_ids):
    df_test.append(test.iloc[600*i : 600*i + 600])

In [100]:
df_agg_test = pd.DataFrame()

for j in range(test_ids):
    df_agg = pd.DataFrame()
    for i in range(3):
        tmp = df_test[j].iloc[i*200 : i*200 + 200].drop('time', axis = 1, inplace = False).groupby('id').agg(['max','min','mean'])
        df_agg = pd.concat([df_agg, tmp], axis = 1)
        
    df_agg_test = df_agg_test.append(df_agg)

In [101]:
df_agg_test

acc_x                         acc_y                         acc_z  \
           max       min      mean       max       min      mean       max   
id                                                                           
3125 -0.275446 -1.564000 -0.989584  0.172556 -0.239870 -0.020038  0.151487   
3126  0.444149 -1.015893 -0.536384  1.708743  0.128756  0.787744 -0.079464   
3127  0.705013 -0.036775  0.476207  1.253612  0.493329  0.908842  0.162760   
3128  0.171789 -1.045889 -0.527978 -0.258476 -0.780307 -0.504071  0.702574   
3129 -0.027340 -1.691371 -0.916738  0.813046 -0.860883  0.012573  0.952330   
...        ...       ...       ...       ...       ...       ...       ...   
3902 -0.328758 -1.601650 -0.963907  1.608399  0.045032  0.523597  0.237183   
3903 -0.228192 -1.632634 -0.910374  0.830672 -0.509494  0.295785  0.521008   
3904 -0.498971 -1.302081 -0.945427  0.422296 -0.443533 -0.014616  0.319565   
3905  1.330448  0.536568  0.946841 -0.024187 -0.580191 -0.349926  0.084615   
3906 -0.508193 -1.266320 -0.905961  0.071348 -0.766078 -0.275239  0.621691   

                                gy_x                               gy_y  \
           min      mean         max         min       mean         max   
id                                                                        
3125 -0.561128 -0.252037   56.953059  -27.668966  -3.283639   96.185341   
3126 -0.856899 -0.450567   62.204081 -148.947565  -1.723688  241.240196   
3127 -0.409663 -0.157099   32.166240  -52.327129  -5.148798   25.697690   
3128 -0.227997  0.264410   34.672903  -50.891987   1.545244  117.490868   
3129 -0.631258  0.167999  343.576408 -345.447240  -7.205195  226.728939   
...        ...       ...         ...         ...        ...         ...   
3902 -0.562072 -0.144141  247.440613 -246.150470   2.306896  152.805526   
3903 -0.573579 -0.099253  197.196100 -114.029374  29.799778   79.733062   
3904 -0.332901  0.018317  152.814053 -214.831199 -12.580745  114.637900   
3905 -0.703792 -0.335912   66.788750  -45.209937   3.165419   92.467431   
3906 -0.375468 -0.044421   73.216034 -145.237944 -43.662334   30.349189   

                                   gy_z                            acc_x  \
             min       mean         max         min       mean       max   
id                                                                         
3125  -61.862128   5.477074   27.030233  -26.785818  -1.022935 -0.407004   
3126  -66.104760  20.541547  139.089357 -112.523057 -18.874450  0.599154   
3127  -38.428781  -2.801515   97.211730  -23.304426   0.746534  1.682591   
3128 -166.945655  -8.527313   83.321416 -101.961357  -3.613914  0.337281   
3129 -181.644595   7.300365  138.130133  -97.833825   9.648396  0.015642   
...          ...        ...         ...         ...        ...       ...   
3902 -199.846704  -2.301501   87.761471 -241.419881 -13.068968 -0.617956   
3903 -124.895338 -13.160317  114.004443 -142.510924  -6.541332 -0.164979   
3904 -142.559610  -3.138103  111.376910  -99.160623   3.379071 -0.085249   
3905  -83.950263   1.717766   55.777955  -51.905231   2.235541  1.313473   
3906  -63.718874 -12.808795   25.432961  -49.716502  -7.610037 -0.383452   

                             acc_y                         acc_z            \
           min      mean       max       min      mean       max       min   
id                                                                           
3125 -1.498606 -0.994907  0.228040 -0.283154 -0.001015  0.236922 -0.573836   
3126 -1.929033 -0.541366  1.319533 -0.200678  0.556989  0.671876 -1.212052   
3127 -0.278200  0.503414  1.392445  0.490982  0.924087  0.168614 -0.426220   
3128 -1.010192 -0.302934 -0.440219 -1.294482 -0.849260  0.561100 -0.469924   
3129 -2.153047 -0.638771  1.562602 -0.504888  0.164811  1.037876  0.046375   
...        ...       ...       ...       ...       ...       ...       ...   
3902 -1.410371 -1.003655  1.473749 -0.515370  0.384248  0.123358 -0.742539   
3903 -1.709527 -0.83

In [102]:
from xgboost import XGBClassifier
from sklearn.metrics import log_loss

In [104]:
y = train_label['label']

In [105]:
xgb = XGBClassifier(n_estimators = 400, learning_rate = 0.1, 
                            max_depth = 6,objective = 'multi:softprob', random_state = 0)

In [116]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = train_test_split(df_agg2, y, test_size = 0.2, random_state = 0)
evals = [(X_test.values, y_test)]

In [117]:
X = df_agg2.values

In [119]:
start = time.time()
xgb.fit(X, y,
            eval_metric = 'mlogloss', 
            eval_set= evals, 
            early_stopping_rounds = 100)

sec = time.time()-start
times = str(datetime.timedelta(seconds = sec)).split('.')

print('소요시간 : ', times)

[0]	validation_0-mlogloss:2.86677
[1]	validation_0-mlogloss:2.50121
[2]	validation_0-mlogloss:2.21854
[3]	validation_0-mlogloss:1.99123
[4]	validation_0-mlogloss:1.79814
[5]	validation_0-mlogloss:1.63860
[6]	validation_0-mlogloss:1.49750
[7]	validation_0-mlogloss:1.37367
[8]	validation_0-mlogloss:1.26377
[9]	validation_0-mlogloss:1.16694
[10]	validation_0-mlogloss:1.08088
[11]	validation_0-mlogloss:1.00259
[12]	validation_0-mlogloss:0.93184
[13]	validation_0-mlogloss:0.86698
[14]	validation_0-mlogloss:0.80885
[15]	validation_0-mlogloss:0.75487
[16]	validation_0-mlogloss:0.70520
[17]	validation_0-mlogloss:0.65916
[18]	validation_0-mlogloss:0.61661
[19]	validation_0-mlogloss:0.57874
[20]	validation_0-mlogloss:0.54232
[21]	validation_0-mlogloss:0.50940
[22]	validation_0-mlogloss:0.47740
[23]	validation_0-mlogloss:0.44864
[24]	validation_0-mlogloss:0.42246
[25]	validation_0-mlogloss:0.39783
[26]	validation_0-mlogloss:0.37414
[27]	validation_0-mlogloss:0.35227
[28]	validation_0-mlogloss:0.3

In [121]:
y_pred = xgb.predict_proba(df_agg_test.values)

In [123]:
y_pred.shape

(782, 61)

In [125]:
submission.iloc[:,1:] = y_pred
submission.to_csv('submission파일/xgb_200T.csv', index = False)

제출 결과값 : 0.89293

## 100타임별 데이터로 features 확장 

In [144]:
def splitTimes(df, Times):
    
    ids = len(df)
    df_t = pd.DataFrame()
    
    for j in range(ids):
        df_agg = pd.DataFrame()
        for i in range(600//Times):
            tmp = df[j].iloc[i*Times : i*Times + Times].drop('time', axis = 1, inplace =False).groupby('id').agg(['max','min','mean'])
            df_agg = pd.concat([df_agg, tmp], axis = 1)
            
        df_t = df_t.append(df_agg)
        
    return df_t

In [126]:
df_agg_100 = pd.DataFrame()

for j in range(ids):
    df_agg = pd.DataFrame()
    for i in range(6):
        tmp = df[j].iloc[i*100 : i*100 + 100].drop('time', axis = 1, inplace = False).groupby('id').agg(['max','min','mean'])
        df_agg = pd.concat([df_agg, tmp], axis = 1)
        
    df_agg_100 = df_agg_100.append(df_agg)

In [147]:
df_agg_100

acc_x                         acc_y                         acc_z  \
           max       min      mean       max       min      mean       max   
id                                                                           
0     1.304609  0.613582  0.932514  0.110430 -0.541029 -0.213235 -0.116636   
1    -0.059166 -0.268818 -0.145122 -0.001413 -0.880830 -0.499247  0.861091   
2     1.219836 -0.975728  0.299019  0.435565 -0.690528 -0.141193  1.088743   
3    -0.679208 -1.339996 -0.940655 -0.145265 -0.513990 -0.269737  0.211275   
4     0.599720 -0.230245  0.164275  0.220760 -2.055076 -0.912313  0.561841   
...        ...       ...       ...       ...       ...       ...       ...   
3120  0.390798 -1.624711  0.035294 -0.323881 -1.268996 -0.704634  0.849966   
3121 -0.513690 -1.575455 -1.083535  0.117965 -0.315516 -0.123205  0.059454   
3122  0.315652 -2.141936 -1.112460 -0.039063 -2.036646 -0.479227  0.803950   
3123  0.164426 -0.929133 -0.355561  1.361144  0.508415  0.894099  0.010090   
3124  0.538809 -0.935231 -0.144236 -0.415634 -0.891517 -0.723623  0.543977   

                                gy_x                               gy_y  \
           min      mean         max         min       mean         max   
id                                                                        
0    -0.672738 -0.348396   22.405416  -34.747874  -2.326346   57.447581   
1     0.425508  0.649430   38.470659 -128.072159 -20.944118   32.432873   
2     0.073846  0.583523   73.525082  -97.687381  -3.363009  202.876776   
3    -0.181537  0.068042  133.050586  -35.200587  27.737092  141.001242   
4    -1.231836  0.091728  140.227116 -356.302649 -27.515184  284.952954   
...        ...       ...         ...         ...        ...         ...   
3120 -0.251821  0.558326   30.301550 -135.245979 -22.385471   79.449094   
3121 -0.345789 -0.143211  120.896134  -21.361227  17.232365   17.152310   
3122 -0.459267  0.220540  400.662632 -316.817335 -14.910199  342.797555   
3123 -0.646863 -0.307133  168.716922 -121.756774 -16.988002  198.033474   
3124  0.038062  0.412735   31.017538  -33.240136  -3.560661   88.243786   

                                   gy_z                            acc_x  \
             min       mean         max         min       mean       max   
id                                                                         
0     -85.887677  -3.381567   37.116542  -37.407136   6.717072  1.343113   
1     -20.821782   3.649729   28.504687  -44.007985  -7.412253 -0.187404   
2    -189.616276  33.703152   55.642836  -30.302375   2.570386  1.041329   
3     -82.537304   8.264129   56.456908  -85.600536 -11.400603 -0.681379   
4     -42.284998  44.737460  162.734781 -171.614590  13.241582  0.096118   
...          ...        ...         ...         ...        ...       ...   
3120  -92.553254  -4.622105   64.366455  -63.388814  11.881463  0.389090   
3121  -14.948522   3.439220   13.296074  -31.463901  -4.424842 -0.446650   
3122 -350.803519 -10.364176  199.711188 -151.559827  17.533713  0.744666   
3123  -68.251262  41.845392  119.832788 -206.580638   0.263684  0.140369   
3124 -134.268304   2.229617   66.437469  -83.882730   2.726218  0.488881   

                             acc_y                         acc_z            \
           min      mean       max       min      mean       max       min   
id                                                                           
0     0.632367  0.946744  0.133165 -0.624113 -0.268362 -0.054876 -0.657399   
1    -2.156208 -0.938035  0.495337 -0.997057 -0.327094  0.558973 -0.522313   
2    -1.142847  0.114581  0.469709 -0.465613 -0.063943  1.027118  0.247685   
3    -1.058604 -0.896781  0.210985 -0.540827 -0.076683  0.410242  0.028097   
4    -0.831264 -0.323075 -0.401405 -1.734513 -0.898814  1.057852 -0.025278   
...        ...       ...       ...       ...       ...       ...       ...   
3120 -1.586209 -0.612395 -0.027536 -1.289257 -0.658129  0.391422 -0.551185   
3121 -1.260301 -0.87

In [143]:
[(i*100, i*100 + 100) for i in range(600//100) ]

[(0, 100), (100, 200), (200, 300), (300, 400), (400, 500), (500, 600)]

In [145]:
df_test_100 = splitTimes(df_test, 100)

In [146]:
df_test_100

acc_x                         acc_y                         acc_z  \
           max       min      mean       max       min      mean       max   
id                                                                           
3125 -0.275446 -1.564000 -1.000219  0.136919 -0.239870 -0.029624  0.151487   
3126  0.444149 -1.015893 -0.404862  1.708743  0.128756  0.893310 -0.079464   
3127  0.705013  0.247982  0.446097  1.252957  0.622445  0.944809  0.052557   
3128 -0.849907 -1.045889 -0.960144 -0.258476 -0.587936 -0.341726  0.344361   
3129 -0.027340 -1.599601 -0.886774  0.813046 -0.860883  0.060669  0.952330   
...        ...       ...       ...       ...       ...       ...       ...   
3902 -0.328758 -1.601650 -0.952883  1.608399  0.045032  0.605502  0.019090   
3903 -0.228192 -1.285814 -0.844972  0.830672 -0.509494  0.375153  0.521008   
3904 -0.702444 -1.302081 -0.912570  0.271173 -0.210304  0.014417  0.319565   
3905  1.330448  0.536568  0.955818 -0.054388 -0.527229 -0.342319  0.084615   
3906 -0.779602 -1.229892 -0.985065  0.071348 -0.386427 -0.147884  0.002158   

                                gy_x                               gy_y  \
           min      mean         max         min       mean         max   
id                                                                        
3125 -0.561128 -0.253863   56.953059  -27.668966  -4.016179   96.185341   
3126 -0.856899 -0.471926   62.204081 -148.947565  -9.917699  241.240196   
3127 -0.395931 -0.183271   24.690371  -49.099601  -1.940878   13.683507   
3128 -0.227997 -0.074063   28.987138  -32.029103   3.076376   27.106675   
3129 -0.631258  0.188811  305.551418 -345.447240 -62.243198  226.728939   
...        ...       ...         ...         ...        ...         ...   
3902 -0.536550 -0.196069  222.609001 -157.295999  -0.650083  131.915912   
3903 -0.573579 -0.167046  197.196100  -37.028574  80.108981   25.769476   
3904 -0.096818  0.108147  152.814053 -110.243213  17.536171  114.637900   
3905 -0.579002 -0.326170   61.184875  -44.694550   4.475058   88.076246   
3906 -0.375468 -0.161386  -21.282996 -145.237944 -72.797694   16.866523   

                                   gy_z                            acc_x  \
             min       mean         max         min       mean       max   
id                                                                         
3125  -46.604496   5.355909   16.278458  -26.785818  -1.862577 -0.484779   
3126  -66.104760  40.162145  139.089357 -103.477781 -12.758876  0.132422   
3127  -38.428781  -7.721650   46.051159  -21.297368  -6.511244  0.672259   
3128 -134.421510  -9.842961   13.497717  -57.792103  -1.852150  0.171789   
3129 -181.644595   8.794902  138.130133  -80.682234  20.702020 -0.170390   
...          ...        ...         ...         ...        ...       ...   
3902 -163.577838   7.295382   50.793587 -241.419881 -25.800108 -0.749983   
3903 -124.895338 -42.260539   92.087589 -142.510924 -27.076872 -0.698838   
3904 -142.559610  -2.511030  111.376910  -99.160623   3.257687 -0.498971   
3905  -82.444911   4.250141   50.820186  -51.905231   3.818253  1.276280   
3906  -41.353762 -10.222955   11.548022  -46.826017  -9.430146 -0.508193   

                             acc_y                         acc_z            \
           min      mean       max       min      mean       max       min   
id                                                                           
3125 -1.553670 -0.978949  0.172556 -0.216888 -0.010452  0.116178 -0.512097   
3126 -0.998487 -0.667907  1.191174  0.339294  0.682178 -0.163297 -0.744198   
3127 -0.036775  0.506317  1.253612  0.493329  0.872876  0.162760 -0.409663   
3128 -0.803196 -0.095812 -0.520524 -0.780307 -0.666416  0.702574  0.358140   
3129 -1.691371 -0.946703  0.481415 -0.584866 -0.035524  0.614538 -0.122484   
...        ...       ...       ...       ...       ...       ...       ...   
3902 -1.424118 -0.974931  0.752465  0.059756  0.441692  0.237183 -0.562072   
3903 -1.632634 -0.97

In [148]:
xgb = XGBClassifier(n_estimators = 400, learning_rate = 0.1, 
                    max_depth = 6, objective = 'multi:softprob', random_state = 0,
                   verbosity = 1)

X_train, X_test , y_train, y_test = train_test_split(df_agg_100, y, test_size = 0.2, random_state = 0)
evals = [(X_test.values, y_test)]


start = time.time()
xgb.fit(df_agg_100.values, y,
            eval_metric = 'mlogloss', 
            eval_set= evals, 
            early_stopping_rounds = 100)

sec = time.time()-start
times = str(datetime.timedelta(seconds = sec)).split('.')

print('소요시간 : ', times)

[0]	validation_0-mlogloss:2.84373
[1]	validation_0-mlogloss:2.47767
[2]	validation_0-mlogloss:2.19437
[3]	validation_0-mlogloss:1.96222
[4]	validation_0-mlogloss:1.76547
[5]	validation_0-mlogloss:1.60308
[6]	validation_0-mlogloss:1.46231
[7]	validation_0-mlogloss:1.33549
[8]	validation_0-mlogloss:1.22367
[9]	validation_0-mlogloss:1.12485
[10]	validation_0-mlogloss:1.03579
[11]	validation_0-mlogloss:0.95627
[12]	validation_0-mlogloss:0.88389
[13]	validation_0-mlogloss:0.81832
[14]	validation_0-mlogloss:0.75956
[15]	validation_0-mlogloss:0.70522
[16]	validation_0-mlogloss:0.65648
[17]	validation_0-mlogloss:0.61124
[18]	validation_0-mlogloss:0.57049
[19]	validation_0-mlogloss:0.53191
[20]	validation_0-mlogloss:0.49730
[21]	validation_0-mlogloss:0.46437
[22]	validation_0-mlogloss:0.43462
[23]	validation_0-mlogloss:0.40748
[24]	validation_0-mlogloss:0.38206
[25]	validation_0-mlogloss:0.35846
[26]	validation_0-mlogloss:0.33716
[27]	validation_0-mlogloss:0.31735
[28]	validation_0-mlogloss:0.2

In [150]:
y_pred = xgb.predict_proba(df_test_100.values)

In [151]:
y_pred.shape

(782, 61)

In [152]:
submission.iloc[:,1:]= y_pred
submission.to_csv('submission파일/xgb_100T.csv',index = False)

제출 결과값 : 0.9074037067